# Scraping the data of Test matches played in the years 1877 - Jan 2022.
Data is scrapped from ESPNCricinfo and stored as CSV files after doing some Data Cleaning and Preparation.

<img src = "https://static.independent.co.uk/2021/02/03/17/newFile-1.jpg?width=600">
<center>Team India scripting one of Test cricket's most epic comebacks.</center>

First we import and install all the required packages.

In [1]:
!pip install jovian --upgrade --quiet

In [2]:
import jovian
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
import re

In [3]:
# Execute this to save new versions of the notebook
jovian.commit(project="test-match-records")

<IPython.core.display.Javascript object>

[jovian] Updating notebook "ash007online/test-match-records" on https://jovian.ai
[jovian] Committed successfully! https://jovian.ai/ash007online/test-match-records


'https://jovian.ai/ash007online/test-match-records'

## Choosing the URLs

We store the ESPNCricinfo Home Page URL: https://stats.espncricinfo.com in the variable `base_url`.<br>
We have the source Webpage: https://stats.espncricinfo.com/ci/content/records/307847.html. <br>
Let's define a function `get_all_year_links()` to get the links of each year from this `page_url`.
<img src = "https://i.imgur.com/ZIV8Txc.png" width = 600>

In [4]:
def get_all_year_links():
    '''
    Function to return the list of all year-wise links
    '''
    
    base_url = "https://stats.espncricinfo.com"
#     base URL
    page_url = "https://stats.espncricinfo.com/ci/content/records/307847.html"
#     first stats page URL

    response = requests.get(page_url)    
    while response.status_code != 200:
        response = requests.get(page_url)
#     download the page and parse it using BeautifulSoup
    base_doc = BeautifulSoup(response.text, 'html.parser')
    
    year_tags = base_doc.find_all('a', {'class' : "QuoteSummary"})
    year_urls = [base_url + year['href'] for year in year_tags]
#     get the year 'a' tags and extract the 'href' attribute to get the year links
    
    return year_urls

Showing how the function works:

In [5]:
get_all_year_links()[-1]
# showing only the last five year links

'https://stats.espncricinfo.com/ci/engine/records/team/match_results.html?class=1;id=2022;type=year'

Now that we have created a function to get all year URLs, we can define a function `get_matches_in_year(year_url)` which takes a `year_url` and returns a Pandas DataFrame with all the matches in that year.

In [6]:
def get_matches_in_year(year_url):
    '''
    Function to get all the Test matches in a calendar year
    
    year_url - URL corresponding to the Calendar year
    '''
#     download the page and parse it
    year_page = requests.get(year_url)
    while year_page.status_code != 200:
        year_page = requests.get(year_url)
    
    year_doc = BeautifulSoup(year_page.text, 'html.parser')
    
#     extract the headers and edit them accordingly
    header_tags = year_doc.find_all('th')
    headers = [header_tags[i].text for i in range(len(header_tags) - 1)]   

    headers.remove("Team 1")
    headers.remove("Team 2")

    headers.insert(0, "Host Team")
    headers.insert(1, "Away Team")
    
#     extract all the matches
    match_tags = year_doc.find_all('tr', {'class': 'data1'})
    
    matches = []
#     to store each match
    for i in range(len(match_tags)):
        match_details = match_tags[i].find_all('td')
        match = [match_details[j].text for j in range(len(match_details) - 1)]
        matches.append(match)
    
#     we don't need the Test Match No., so we skip it (use len() - 1)
    
#     return a DataFrame with the data and headers
    return pd.DataFrame(matches, columns = headers)

The function works as follows:

In [7]:
get_matches_in_year(get_all_year_links()[-1])
# sending the last link of the list

,Host Team,Away Team,Winner,Margin,Ground,Match Date
0,New Zealand,Bangladesh,Bangladesh,8 wickets,Mount Maunganui,"Jan 1-5, 2022"
1,South Africa,India,South Africa,7 wickets,Johannesburg,"Jan 3-6, 2022"
2,Australia,England,drawn,,Sydney,"Jan 5-9, 2022"
3,New Zealand,Bangladesh,New Zealand,inns & 117 runs,Christchurch,"Jan 9-11, 2022"
4,South Africa,India,South Africa,7 wickets,Cape Town,"Jan 11-14, 2022"
5,Australia,England,Australia,146 runs,Hobart,"Jan 14-16, 2022"


#### Special Case of Pakistan
Unlike most other Test-playing Nations, Pakistan has been forced to hold its Home games outside Pakistan (most notably in UAE) due to security reasons. We need to handle such special cases. 
<br>We define two functions - `host_away_swapper(matches, matches_df)` and `host_correction(matches)` to do so. The documentation of each function is given below.

In [8]:
def host_away_swapper(matches, matches_df):
    '''
    Helper function to swap the "Host Team" and "Away Team" columns
    
    matches_df - extracted DataFrame whose column entries will be swapped
    matches - DataFrame with all the matches to which the swapped data will be finally assigned to
    '''
    
    for i in matches_df.index:
        matches_df.loc[i, "Host Team"], matches_df.loc[i, "Away Team"] = matches_df.loc[i, "Away Team"], matches_df.loc[i, "Host Team"] 
#     do the swapping
    
    matches.loc[matches_df.index] = matches_df
#     make the changes in the original Dataframe and return it
    return matches

In [9]:
def host_correction(matches):
    '''
    Helper Function to correct the host countries discrepancies
    
    matches - DataFrame containing all the matches
    '''
    
#     matches held in UAE and hosted by Pakistan Cricket Board
    UAE_grounds = ["Abu Dhabi", "Dubai (DSC)", "Sharjah"]
    for ground in UAE_grounds:
        ground_match = matches[matches["Ground"] == ground]
        ground_match = ground_match[ground_match["Away Team"] == "Pakistan"]
#         DataFrame extracted
        matches = host_away_swapper(matches, ground_match)
#         changes made in the original

    aus_vs_pak = matches[(matches["Host Team"] == "Australia") & (matches["Away Team"] == "Pakistan")]
    aus_vs_pak_grounds = ["Colombo (PSS)", "Lord's", "Leeds"]
#         matches of the 2002 and 2010 Pak vs Aus series held outside Pakistan (but not in UAE), 
#         hosted by Pakistan Cricket Board
    
    for ground in aus_vs_pak_grounds:
        ground_match = aus_vs_pak[aus_vs_pak["Ground"] == ground]
        
        matches = host_away_swapper(matches, ground_match)
#         changes made in original

    return matches

The function `host_correction(...)` works as follows:

In [10]:
matches = get_matches_in_year(get_all_year_links()[-5])
matches[matches["Away Team"] == "Pakistan"]

,Host Team,Away Team,Winner,Margin,Ground,Match Date
12,Ireland,Pakistan,Pakistan,5 wickets,Dublin (Malahide),"May 11-15, 2018"
13,England,Pakistan,Pakistan,9 wickets,Lord's,"May 24-27, 2018"
14,England,Pakistan,England,inns & 55 runs,Leeds,"Jun 1-3, 2018"
29,Australia,Pakistan,drawn,,Dubai (DSC),"Oct 7-11, 2018"
31,Australia,Pakistan,Pakistan,373 runs,Abu Dhabi,"Oct 16-19, 2018"
36,New Zealand,Pakistan,New Zealand,4 runs,Abu Dhabi,"Nov 16-19, 2018"
39,New Zealand,Pakistan,Pakistan,inns & 16 runs,Dubai (DSC),"Nov 24-27, 2018"
41,New Zealand,Pakistan,New Zealand,123 runs,Abu Dhabi,"Dec 3-7, 2018"
47,South Africa,Pakistan,South Africa,6 wickets,Centurion,"Dec 26-28, 2018"


As we see in the entries 29, 31,..., 41, the matches are held in UAE grounds, but Pakistan is set as the `Away Team`. Hence we use the host_correction(...) method and show the entries again.

In [11]:
matches = host_correction(matches)
matches[matches["Host Team"] == "Pakistan"]

,Host Team,Away Team,Winner,Margin,Ground,Match Date
29,Pakistan,Australia,drawn,,Dubai (DSC),"Oct 7-11, 2018"
31,Pakistan,Australia,Pakistan,373 runs,Abu Dhabi,"Oct 16-19, 2018"
36,Pakistan,New Zealand,New Zealand,4 runs,Abu Dhabi,"Nov 16-19, 2018"
39,Pakistan,New Zealand,Pakistan,inns & 16 runs,Dubai (DSC),"Nov 24-27, 2018"
41,Pakistan,New Zealand,New Zealand,123 runs,Abu Dhabi,"Dec 3-7, 2018"


As we can infer, the helper function works perfectly.

## Strategy

Now that we have functions to perform web scraping on the main records link and all the individual year links, we need to clean and prepare the data to save it into CSV files.

We can scrape the matches and store them as a single CSV file with all the matches or separate them:
1. Year-wise,
2. Team-wise,
3. Hosting nation-wise, and
4. Ground-wise 

and then store them as multiple CSV files.

All the subsequent sub-folders shall be stored in a single "Test Matches" folder.<br>
Let's do them one by one:

### Year-wise

Let's define a function `create_year_matches_csv(year_url, path)` which takes the URL of a year and stores the matches of that year in a CSV file at the specified `path`.

In [12]:
def create_year_matches_csv(year_url, path):
    '''
    Function to create a CSV file from the data in year_url and store the file at path
    '''
    
    year_df = host_correction(get_matches_in_year(year_url))
#     use the functions defined above to get the DataFrame with the matches

    year_df.to_csv(path, index = None)
#     store as CSV 

Now define a `create_years_folder(year_urls)` function which takes the list of all year URLs and uses the `create_year_matches_csv(...)` function to create CSV files for each URL in the appropriate directory.

In [13]:
def create_years_folder(year_urls):
    '''
    Function to extract all the test matches, year-wise, and store them as CSV files, in a "By Year" folder
    
    year_urls - list of year-wise data URLs
    '''
    
    os.makedirs("Test Matches/By Year", exist_ok = True)
#     create the folder

    for year in year_urls:
        yr = year.split(';')[1][-4:]
#         get the year
        year_df = create_year_matches_csv(year, 'Test Matches/By Year/{}.csv'.format(yr))
#         use the create_year_matches_csv to store the data for each year 

Now we can use the `get_all_year_links()` function to get the URLs of all the years and use the `create_years_folder(...)` function to create a folder with all the years' matches as a CSV file.<br>
The function will be used later in a mega function to perform all the Folder creation.

### Collecting all matches

We observe that by the time we have created the "By Year" folder, we have performed the web scraping.<br>
Invoking the `get_matches_in_year(...)` function again and again would be a waste of time. What we can do is:
* Get the list of CSV files in "By Year" folder
* Create an empty DataFrame
* Read the contents of the first CSV file into a DataFrame
* Append the contents DataFrame to the end of the empty DataFrame
* Repeat the process of reading and appending for all the files

We thus get a DataFrame with all the already scrapped matches. We need to perform minor `host_correction(...)` and drop the duplicate entries in the DataFrame to get the required DataFrame of `all_matches`.
<br></br>
We define a function `collect_all_matches()` to code the algorithm explained above.

In [14]:
def collect_all_matches():
    '''
    Function to parse the data from all CSV files, and return all matches in a single DataFrame
    '''
    
    year_files = sorted(os.listdir("Test Matches/By Year"))
#     get the list of files from which matches have to be scrapped
    
    all_matches = pd.DataFrame()
#     creating an empty DataFrame which will store the final data

    for year in year_files:
        year_df = pd.read_csv("Test Matches/By Year/{}".format(year), na_filter = False)
#         read data from each CSV file
        all_matches = all_matches.append(year_df).reset_index(drop = True)
#         and append the data to the all_matches DataFrame

    all_matches = host_correction(all_matches)
#     perform the host corrections for special cases
    
    all_matches.drop_duplicates(inplace = True, ignore_index = True)    
#     this removes any and every duplicate match entry in the DataFrame
    
    return all_matches
#     return the final DataFrame

***Note: `collect_all_matches()` will work ONLY and ONLY IF the `create_years_folder(...)` function is first called, as the former function is dependent on the outcome of the latter function.***

### Team-wise

Let's define a function `create_team_matches_csv(all_matches, team)` which takes the DataFrame containing all matches and a team name, and creates a CSV file containing the matches featuring the passed `team`.

In [15]:
def create_team_matches_csv(all_matches, team):
    '''
    Function to create a CSV file containing matches played by a team.
    
    all_matches - DataFrame of all Test matches
    team - name of team
    '''
    
    team_matches = all_matches[(all_matches["Host Team"] == team) | (all_matches["Away Team"] == team)]
#     matches extracted

    team_matches.to_csv("Test Matches/By Team/{}.csv".format(team), index = None)
#     DataFrame saved as CSV file in appropriate location with appropriate filename

Next we define a function `create_teams_folder(all_team_matches)` which creates a folder "By Team" containing CSV files for each individual team.

In [16]:
def create_teams_folder(all_team_matches):
    '''
    Function to create the Team-wise CSV files.
    
    all_team_matches - DataFrame containing matches of all teams
    '''
    
    teams = all_team_matches["Host Team"].unique()
#     get the names of each team
    
    os.makedirs("Test Matches/By Team", exist_ok = True)
#     create a new directory
    
    for team in teams:
        create_team_matches_csv(all_team_matches, team)
#     call the helper function to create CSV files at location

Now we can use the `get_all_year_links()` function to get the URLs of all the years and use the `create_teams_folder(...)` function to create a folder with all the matches of each team as a CSV file.<br>
The function will be used later in a mega function to perform all the Folder creation.

### Hosting nation-wise

**This section requires some extra cleaning steps.<br>**

There are total 5 matches which have been played effectively at a neutral venue, i.e., the match was played in neither of the two competing nations.<br>
The matches include: 
* 3 Australia vs South Africa matches, part of the 1912 Triangular Test Series, played in England
* 1 Pakistan vs Sri Lanka match, final of 1999 Asian Test Championship, played in Dhaka, Bangladesh
* 1 India vs New Zealand match, final of 2019-2021 World Test Championship, played in Southampton, England

These matches will be listed under the countries where the match was played.<br></br>
***Note: Matches organised by Pakistan Cricket Board and Afghanistan Cricket Board but played outside Pakistan and Afghanistan will still be listed under Pakistan.***

We need two functions - `neutral_venue_encoder(...)` and `neutral_venue_decoder(...)` to enable the Pandas slicing to effectively fulfill the conditions mentioned above, as well as maintain the correct pattern while creating the CSV file.

In [17]:
def neutral_venue_encoder(all_matches, host, away, ground, country):
    '''
    Helper Function to encode the matches at a neutral venue appropriately, enabling proper Pandas slicing with the country
    
    all_matches - DataFrame containing all the matches
    host - original host country
    away - original away country
    ground - ground at which the match happened
    country - country to be set as the new temporary host
    '''
    
    matches = all_matches[(all_matches["Host Team"] == host) & (all_matches["Away Team"] == away) & (all_matches["Ground"] == ground)]
#     get the neutral matches
    i = matches.index
    matches.loc[i, "Host Team"] = country
    matches.loc[i, "Away Team"] = host+ " " +away
#     perform the encoding

    all_matches.loc[matches.index] = matches
#     make the changes in the original DataFrame and return it
    return all_matches

In [18]:
def neutral_venue_decoder(country_matches_df, host, away):
    '''
    Helper Function to decode the match at neutral venue back to its original format
    
    country_matches_df - DataFrame with all the matches
    host - Team 1, in this case
    away - Team 2, in this case
    '''
    
    matches = country_matches_df[country_matches_df["Away Team"] == host+ " " +away]
#     get the encoded matches
    
    for i in matches.index:
        matches.loc[i, "Host Team"], matches.loc[i, "Away Team"] = host, away
#     do the decoding

    country_matches_df.loc[matches.index] = matches
#     make the changes in the original DataFrame and return it
    return country_matches_df

As you may have guessed, the above functions work for each special case, so we define another helper function `neutral_matches_segregate(all_matches)` which uses the `neutral_venue_encoder(...)` function and returns the Master DataFrame with all changes.

In [19]:
def neutral_matches_segregate(all_matches):
    '''
    Minor Helper function to encode the 5 special neutral matches
    
    all_matches - DataFrame of all test matches
    '''
    
    aus_vs_sa_grounds = ["Lord's", "Nottingham", "Manchester"]
#     matches of the 1912 Triangular series b/w Eng, Aus and SA, all of which were played in England
    for ground in aus_vs_sa_grounds:
        all_matches = neutral_venue_encoder(all_matches, "Australia", "South Africa", ground, "England")
#         changes made directly by calling helper function

    all_matches = neutral_venue_encoder(all_matches, "India", "New Zealand", "Southampton", "England")
#     2021 WTC Final held in England
    all_matches = neutral_venue_encoder(all_matches, "Pakistan", "Sri Lanka", "Dhaka", "Bangladesh")
#     1999 ATC Final held in Bangladesh

    return all_matches

Let's define a function `create_host_matches_csv(all_matches, nation)` which takes the DataFrame containing all matches and a team name, and creates a CSV file for the matches hosted by the passed `nation`.

In [20]:
def create_host_matches_csv(all_matches, nation):
    '''
    Function to extract the matches of the country mentioned in path and store as a CSV file
    
    all_matches - DataFrame with all Test matches
    nation - hosting nation 
    '''
    
    nation_matches_df = all_matches[all_matches["Host Team"] == nation]
#     extract the matches in the country
    
    if nation == "England":    
        nation_matches_df = neutral_venue_decoder(nation_matches_df, "Australia", "South Africa")
        nation_matches_df = neutral_venue_decoder(nation_matches_df, "India", "New Zealand")
#         2021 WTC final and 1912 Triangular series matches

    if nation == "Bangladesh":
        nation_matches_df = neutral_venue_decoder(nation_matches_df, "Pakistan", "Sri Lanka")
#         1999 ATC final
    
    nation_matches_df.to_csv("Test Matches/By Hosting Nation/{}.csv".format(nation), index=None)
#     store the data as CSV

Next we define a function `create_hosts_folder(all_host_matches)` which creates a folder "By Hosting Nation" containing CSV files for each Hosting Nation.

In [21]:
def create_hosts_folder(all_host_matches):
    '''
    Function to create the Hosts Folder having the data for each Host country
    
    all_host_matches - DataFrame containing matches hosted by all hosts
    '''
    
    all_host_matches = neutral_matches_segregate(all_host_matches)
#     minor correction of data using the helper functions

    hosts = all_host_matches["Host Team"].unique()
#     extract the names of Hosts

    os.makedirs("Test Matches/By Hosting Nation", exist_ok = True)
#     create the folder

    for nation in hosts:
        create_host_matches_csv(all_host_matches, nation)
#         call the function to create the CSV file in folder

Now we can use the `get_all_year_links()` function to get the URLs of all the years and use the `create_hosts_folder(...)` function to create a folder with all the matches hosted by a nation('s Board) as a CSV file.<br>
The function will be used later in a mega function to perform all the Folder creation.

### Ground-wise

In this case, it would be more convenient to have sub-folders for each countries, and each ground in a country have its own CSV file. <br></br>
Hence we define a function `country_name(ground_matches)` which returns the name of the country in which the ground is located.<br>The function also handles special cases such as grounds in UAE.

In [22]:
def country_name(ground_matches):
    '''
    Helper function to return the country of the ground
    
    ground_matches - DataFrame containing all the matches played at a ground
    '''
    
    ground = ground_matches.Ground.values[0]
#     get the ground name in a string

    UAE = ["Abu Dhabi", "Dubai (DSC)", "Sharjah"]
#     special case for grounds of UAE

    if ground in UAE:
        return "UAE"
    if ground == "Dehradun":
        return "India"
#     special case - Dehradun, in India, hosted a single game between Afghanistan and Ireland

    return ground_matches["Host Team"].value_counts().sort_values().index[-1]
#     return the team which hosted the most games at the venue (general observation)

Let's define a function `create_ground_matches_csv(all_matches, ground)` which takes the DataFrame containing all matches and a ground name, and creates a CSV file containing all matches played at the passed `ground`.

In [23]:
def create_ground_matches_csv(all_matches, ground):
    '''
    Function to create a CSV file for each ground
    
    all_matches - DataFrame of all matches
    ground - ground for which CSV file is to be created
    '''
    
    ground_matches = all_matches[all_matches.Ground == ground]
#         get all the matches at the ground

    country = country_name(ground_matches)
#     use the helper function to get the country name, and create a new subfolder for that country
    os.makedirs("Test Matches/By Ground/{}".format(country), exist_ok = True)
        
    ground_matches.to_csv("Test Matches/By Ground/{}/{}.csv".format(country, ground), index = None)
#         write each ground_matches DataFrame to a CSV file at the appropriate location

Next we define a function `create_grounds_folder(all_ground_matches)` which creates a folder "By Ground" containing CSV files for each ground, in a sub-folder having the name of the country in which the ground exists.

In [24]:
def create_grounds_folder(all_ground_matches):
    '''
    Function to create a By Ground folder, with the location folders
    
    all_ground_matches - DataFrame containing matches played at all grounds
    '''

    grounds = all_ground_matches.Ground.unique()
#     extracting list of all grounds

    os.makedirs("Test Matches/By Ground", exist_ok = True)
# create a new folder "By Ground"    
    
    for ground in grounds:
#     for each ground, use helper function to create a CSV file
        create_ground_matches_csv(all_ground_matches, ground)

Now we can use the `get_all_year_links()` function to get the URLs of all the years and use the `create_grounds_folder(...)` function to create a folder having country folders with all the matches played at a ground as a CSV file.<br>
The function will be used later in a mega function to perform all the Folder creation.

## Main Function

Once we created the functions for both scraping the data from the webpages and saving them in an organised manner, we can create a `main` function which will perform all of the above in one call.

Let's define the `main` function.

In [25]:
def main():
    '''
    Main Function which performs web scraping and organised saving to CSV files.
    '''
    
    os.makedirs("Test Matches", exist_ok = True)
#     create a head directory to store all the data

    links = get_all_year_links()
#     get the list of URLs of all the webpages to be scrapped
    
    create_years_folder(links)
#     scrape the webpages and store the data in year-wise CSV files

    matches = collect_all_matches()
#     read the created CSV files and collect all matches in a single DataFrame

    create_teams_folder(matches)
#     use the DataFrame to create team-wise CSV files
    create_hosts_folder(matches)
#     use the DataFrame to create hosting nation-wise CSV files
    create_grounds_folder(matches)
#     use the DataFrame to create ground-wise CSV files

    matches.to_csv("Test Matches/All Matches.csv", index = None)
#     in the end, store all matches in a single CSV file in the head directory

Call the `main()` function to do the job. 

In [ ]:
main()

## References and Future Work

Resources include:
* <a href = "https://stats.espncricinfo.com/ci/engine/records/index.html"> ESPNCricinfo Stats </a>
* <a href = "https://www.geeksforgeeks.org/"> Geeksforgeeks</a>
* <a href = "https://pandas.pydata.org/"> Pandas</a>
* <a href = "https://stackoverflow.com/"> Stack Overflow</a>
* <a href = "https://www.youtube.com/watch?v=RKsLLG-bzEY"> YouTube Video on Web Scraping by Jovian </a>
* <a href = "https://jovian.ai"> Jovian </a>

Now we have successfully scrapped the data and created a dataset.<br>
* The dataset can be updated by simply running the `main()` function as and when required.
* The dataset can be expanded to have more details like : link to scorecard of the match, Man of the Match, etc.
* The dataset can be used to perform Exploratory Data Analysis and Predictive Analysis.
* The dataset can be further expanded to include ODI matches and T20I matches.
* Web scraping can be performed to get the data for ODI and T20I matches.

Hope you liked my work. Efforts will be made to include the data for ODIs and T20Is in future.<br>
Until then, ***Long Live Test Cricket***.

In [ ]:
jovian.commit(outputs = "Test Matches")
# the entire Folder will be uploaded to the Jovian portal